<a href="https://colab.research.google.com/github/fadel11-hub/Tim-Coba-Coba/blob/main/Final_Model_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Whisper

In [2]:
!pip install openai-whisper

# Klasifikasi

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# --- 1. Load dataset ---
df = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")  # ganti dengan file datasetmu
texts = df["text"].astype(str).values
labels = df["label"].values

# --- 2. Encode label ke angka ---
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# --- 3. Tokenisasi dan padding ---
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_length = 20  # panjang maksimum kalimat
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

vocab_size = len(tokenizer.word_index) + 1  # +1 untuk padding
embedding_dim = 50

print("\n--- Hasil Tokenisasi dan Padding ---")
print(f"Shape Padded Sequences: {padded_sequences}")
print(f"Shape Encoded Labels: {encoded_labels}")



--- Hasil Tokenisasi dan Padding ---
Shape Padded Sequences: [[  9   2   4 ...   0   0   0]
 [ 38   2   2 ...   0   0   0]
 [  5   6  39 ...   0   0   0]
 ...
 [  2  23 254 ...   0   0   0]
 [  2  22  37 ...   0   0   0]
 [  2 257  76 ...   0   0   0]]
Shape Encoded Labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [4]:
# --- 4. Split data ---
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, encoded_labels, test_size=0.25, random_state=42
)

print("\n--- Hasil Pembagian Data ---")
print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test (Validation): {X_test.shape}")
print(f"Shape y_test (Validation Labels): {y_test.shape}")


--- Hasil Pembagian Data ---
Shape X_train: (153, 20)
Shape y_train: (153,)
Shape X_test (Validation): (51, 20)
Shape y_test (Validation Labels): (51,)


In [5]:
# --- 5. Bangun model LSTM ---
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# --- 6. Latih model ---
history = model.fit(
    X_train, y_train, epochs=10, batch_size=16,
    validation_data=(X_test, y_test)
)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 137ms/step - accuracy: 0.4875 - loss: 0.6926 - val_accuracy: 0.4314 - val_loss: 0.6952
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.5504 - loss: 0.6914 - val_accuracy: 0.4314 - val_loss: 0.6973
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.5214 - loss: 0.6923 - val_accuracy: 0.4314 - val_loss: 0.6972
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5441 - loss: 0.6914 - val_accuracy: 0.4314 - val_loss: 0.6965
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.5471 - loss: 0.6888 - val_accuracy: 0.4314 - val_loss: 0.6978
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5658 - loss: 0.6853 - val_accuracy: 0.9412 - val_loss: 0.6513
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.8950 - loss: 0.5661 - val_accuracy: 0.9216 - val_loss: 0.2325
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.9957 - loss: 0.0927 - val_accuracy: 0.8824

# Evaluasi

In [7]:

# --- 7. Evaluasi model ---
loss, accuracy = model.evaluate(X_test, y_test)

print("\n--- Hasil Evaluasi Model ---")
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.9165 - loss: 0.4405

--- Hasil Evaluasi Model ---
Loss: 0.4155
Accuracy: 0.9216
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step

--- Classification Report ---
              precision    recall  f1-score   support

        aman       0.85      1.00      0.92        22
     darurat       1.00      0.86      0.93        29

    accuracy                           0.92        51
   macro avg       0.92      0.93      0.92        51
weighted avg       0.93      0.92      0.92        51



In [8]:
# --- 7. Simpan model dan tokenizer ---
model.save("lstm_model.h5")
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [9]:
import whisper

model_whisper = whisper.load_model("base")

result3 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Tolong aku.m4a")
result4 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Saya menonton video.m4a")
result5 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat")
result6 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/aman")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [10]:
from tensorflow.keras.models import load_model
import pickle

# Load model & tokenizer
model = load_model("lstm_model.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Kalimat baru
seq = tokenizer.texts_to_sequences(result3['text'])
pad = pad_sequences(seq, maxlen=20, padding="post")

# Prediksi
pred_prob = model.predict(pad)[0][0]
pred_label = 1 if pred_prob >= 0.5 else 0
final_label = label_encoder.inverse_transform([pred_label])[0]

print("Kalimat:", result4['text'])
print("Probabilitas darurat:", pred_prob)
print("Prediksi Label:", final_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
Kalimat:  I am Manantan Video.
Probabilitas darurat: 0.004186824
Prediksi Label: aman


# Dependency

In [11]:
print('whisper', whisper.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('tensorflow', tf.__version__)
print('scikit-learn', sklearn.__version__)

whisper 20250625
pandas 2.2.2
numpy 2.0.2
tensorflow 2.18.0
scikit-learn 1.6.1


In [12]:
!pip freeze > requirements.txt

In [13]:

%%writefile requirements.txt
whisper=20250625
pandas=2.2.2
numpy=2.0.2
tensorflow=2.18.0
scikit-learn=1.6.1


Overwriting requirements.txt


In [14]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>